This notebook contains examples of using model index and OPC UA functions to download inverters, strings set and trackers data of the site 'EG-AS' from the model index and OPC UA api servers. The aggregated historical data downloaded in the data folder and cell execution time can be noted for each request.

### Import Libraries

In [1]:
%load_ext autotime

time: 454 µs (started: 2022-08-26 15:23:33 +02:00)


In [2]:
# Import the required packeages
import pandas as pd
import urllib3
urllib3.disable_warnings()
from pandas.io.json import json_normalize
from typing import List, Dict
import sys
import os
import numpy as np
import concurrent.futures
import json
import itertools
from aiohttp import ClientSession
import asyncio
import dask.dataframe as dd

time: 575 ms (started: 2022-08-26 15:23:33 +02:00)


### Import Scripts

In [3]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/meena/dev/migrate/pyPrediktorMapClient/src/pyprediktormapclient
time: 1.12 ms (started: 2022-08-26 15:23:34 +02:00)


In [4]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

time: 78.8 ms (started: 2022-08-26 15:23:34 +02:00)


In [5]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opcua_rest_url = "http://10.241.68.86:13371/"
opcua_server_url = "opc.tcp://10.241.80.4:4872"

# Model index API
mdx = ModelIndex(url=model_index_url)

# OPC UA API
opc = OPC_UA(rest_url=opcua_rest_url, opcua_url= opcua_server_url)

time: 63.5 ms (started: 2022-08-26 15:23:34 +02:00)


### OPC UA server parameters

In [6]:
# Input parameters for value data

# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z"  
end_time = "2022-07-06T07:55:14.544000Z" # 5th July aggregated data
end_time1 = "2022-07-20T07:55:14.544000Z" # 15 days
end_time2 = "2022-08-05T07:55:14.544000Z" # 30 days
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

time: 540 µs (started: 2022-08-26 15:23:34 +02:00)


In [7]:
# All the sites on the OPC server
sites = mdx.get_objects_of_type('SiteType')

# Selecting specific site 
sitename = 'EG-AS'
site = sites[sites['DisplayName'] == sitename]

time: 46.9 ms (started: 2022-08-26 15:23:34 +02:00)


INVERTERS DATA

In [8]:
# All the inverters on the site
inverters = mdx.get_object_descendants("InverterType", site, "PV_Assets")
inverters

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Vars,Props
0,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19,EG-AS-TS10-I19,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
1,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I20,EG-AS-TS10-I20,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
2,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS11.I21,EG-AS-TS11-I21,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
3,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS11.I22,EG-AS-TS11-I22,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
4,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z2.TS09.I17,EG-AS-TS09-I17,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
5,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z2.TS09.I18,EG-AS-TS09-I18,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
6,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS03.I05,EG-AS-TS03-I05,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
7,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS03.I06,EG-AS-TS03-I06,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
8,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS06.I11,EG-AS-TS06-I11,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."
9,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z3.TS06.I12,EG-AS-TS06-I12,InverterType,EG-AS,"[{'DisplayName': 'InverterDCLosses', 'Id': '4:...","[{'DisplayName': 'GPSLatitude', 'Value': '0.0'..."


time: 185 ms (started: 2022-08-26 15:23:34 +02:00)


In [9]:
# List of unique inverter variables
inv_var_list = ['ACActivePower', 'ACCurrentAverage', 'ACCurrentPhaseA', 'ACCurrentPhaseB', 'ACCurrentPhaseC', 'ACDailyActiveEnergy', 'ACReactivePower', 'ACTotalActiveEnergy', 
'ACVoltagePhaseAB', 'ACVoltagePhaseAC', 'ACVoltagePhaseBC', 'DCCurrent', 'DCPower', 'DCVoltage', 'Frequency', 'PowerFactor', 'Status']

time: 1.04 ms (started: 2022-08-26 15:23:34 +02:00)


In [10]:
# Live value inverters data
inv_liv_values = opc.get_live_values_data(inv_var_list, inverters)
inv_liv_values

,Id,Name,Variable,Timestamp,Value,Code,Quality
0,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACActiveP...,EG-AS-TS10-I19,ACActivePower,2022-08-26T13:22:23.8039959Z,2.289300e+06,NaN,NaN
1,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentAverage,2022-08-26T13:22:23.8039959Z,2.006333e+03,NaN,NaN
2,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentPhaseA,2022-08-26T13:22:23.8039959Z,2.007000e+03,NaN,NaN
3,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentPhaseB,2022-08-26T13:22:23.8039959Z,2.007600e+03,NaN,NaN
4,4:1:SSO.EG-AS.S1.Z1.TS10.I19.Signals.ACCurrent...,EG-AS-TS10-I19,ACCurrentPhaseC,2022-08-26T13:22:23.8039959Z,2.004400e+03,NaN,NaN
...,...,...,...,...,...,...,...
369,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.DCPower,EG-AS-TS05-I10,DCPower,2022-08-26T13:22:23.8039959Z,2.148600e+06,NaN,NaN
370,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.DCVoltage,EG-AS-TS05-I10,DCVoltage,2022-08-26T13:22:23.8039959Z,1.024350e+03,NaN,NaN
371,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.Frequency,EG-AS-TS05-I10,Frequency,2022-04-27T10:17:52.428Z,0.000000e+00,2.147484e+09,Bad
372,4:1:SSO.EG-AS.S1.Z6.TS05.I10.Signals.PowerFactor,EG-AS-TS05-I10,PowerFactor,2022-04-27T10:17:52.428Z,0.000000e+00,2.147484e+09,Bad


time: 203 ms (started: 2022-08-26 15:23:35 +02:00)


In [11]:
# 1 day aggregated historical inverter data
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time, pro_interval, agg_name, inverters, inv_var_list)
await session.close()

time: 5.1 s (started: 2022-08-26 15:23:35 +02:00)


In [12]:
# 15 day inverters data
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time1, pro_interval, agg_name, inverters, inv_var_list)
await session.close()

time: 20.9 s (started: 2022-08-26 15:23:40 +02:00)


In [14]:
# 1 month inverter data 
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time2, pro_interval, agg_name, inverters, inv_var_list)
await session.close()

time: 42.2 s (started: 2022-08-26 15:26:24 +02:00)


STRINGS DATA

In [15]:
# Strings set data 
strings = mdx.get_object_descendants("StringSetType", site, "PV_Assets")
strings

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Vars,Props
0,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01,EG-AS-TS10-I19-SM01-CH01,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '1'},..."
1,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02,EG-AS-TS10-I19-SM01-CH02,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '2'},..."
2,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH03,EG-AS-TS10-I19-SM01-CH03,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '3'},..."
3,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH04,EG-AS-TS10-I19-SM01-CH04,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '4'},..."
4,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH05,EG-AS-TS10-I19-SM01-CH05,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '5'},..."
...,...,...,...,...,...,...,...
2933,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH06,EG-AS-TS05-I10-SM13-CH06,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '126'..."
2934,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH07,EG-AS-TS05-I10-SM13-CH07,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '127'..."
2935,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH08,EG-AS-TS05-I10-SM13-CH08,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '128'..."
2936,4:1:SSO.EG-AS,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09,EG-AS-TS05-I10-SM13-CH09,StringSetType,EG-AS,"[{'DisplayName': 'Height', 'Id': '4:1:SSO.EG-A...","[{'DisplayName': 'ColPosition', 'Value': '129'..."


time: 652 ms (started: 2022-08-26 15:27:10 +02:00)


In [16]:
# List of strings variables
strng_var_list = ['DCCurrent', 'DCPower', 'DCVoltage', 'RelativeDCPower']

time: 940 µs (started: 2022-08-26 15:27:12 +02:00)


In [17]:
# Live value strings data 
strng_liv_values = opc.get_live_values_data(strng_var_list, strings)
strng_liv_values

,Id,Name,Variable,Timestamp,Value,Code,Quality
0,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Signals...,EG-AS-TS10-I19-SM01-CH01,DCCurrent,2022-08-26T13:26:23.8039959Z,17.319000,NaN,NaN
1,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Signals...,EG-AS-TS10-I19-SM01-CH01,DCPower,2022-08-26T13:26:23.8039959Z,17946.814000,NaN,NaN
2,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Signals...,EG-AS-TS10-I19-SM01-CH01,DCVoltage,2022-08-26T13:26:23.8039959Z,1036.250000,NaN,NaN
3,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01.Signals...,EG-AS-TS10-I19-SM01-CH01,RelativeDCPower,2022-08-26T13:26:23.8039959Z,0.842573,NaN,NaN
4,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02.Signals...,EG-AS-TS10-I19-SM01-CH02,DCCurrent,2022-08-26T13:26:23.8039959Z,17.216000,NaN,NaN
...,...,...,...,...,...,...,...
11747,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09.Signals...,EG-AS-TS05-I10-SM13-CH09,RelativeDCPower,2022-08-26T13:26:23.8039959Z,0.771728,NaN,NaN
11748,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH10.Signals...,EG-AS-TS05-I10-SM13-CH10,DCCurrent,2022-08-26T13:26:23.8039959Z,16.492000,NaN,NaN
11749,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH10.Signals...,EG-AS-TS05-I10-SM13-CH10,DCPower,2022-08-26T13:26:23.8039959Z,16881.210000,NaN,NaN
11750,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH10.Signals...,EG-AS-TS05-I10-SM13-CH10,DCVoltage,2022-08-26T13:26:23.8039959Z,1023.600000,NaN,NaN


time: 12.9 s (started: 2022-08-26 15:27:14 +02:00)


In [18]:
# 1 day strings data
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time, pro_interval, agg_name, strings, strng_var_list)
await session.close()

time: 1min 18s (started: 2022-08-26 15:27:30 +02:00)


In [27]:
# 15 days strings data
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time1, pro_interval, agg_name, strings, strng_var_list)
await session.close()

TypeError: 'NoneType' object is not iterable

time: 17min 1s (started: 2022-08-26 14:15:05 +02:00)


Code run, downloaded four parquet files in the data folder and failed.

In [28]:
# 1 month strings data
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time2, pro_interval, agg_name, strings, strng_var_list)
await session.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4147914d30>


TRACKERS DATA

In [19]:
# Trackers data 
trackers = mdx.get_object_ancestors("TrackerType", strings, "PV_Serves")
trackers

,ObjectId,AncestorId,AncestorName,AncestorType,ObjectName,Vars,Props
0,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH01,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR001,EG-AS-TR-TB10.TR001,TrackerType,EG-AS-TS10-I19-SM01-CH01,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
1,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH02,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR002,EG-AS-TR-TB10.TR002,TrackerType,EG-AS-TS10-I19-SM01-CH02,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
2,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH03,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR004,EG-AS-TR-TB10.TR004,TrackerType,EG-AS-TS10-I19-SM01-CH03,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
3,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH04,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR003,EG-AS-TR-TB10.TR003,TrackerType,EG-AS-TS10-I19-SM01-CH04,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
4,4:1:SSO.EG-AS.S1.Z1.TS10.I19.SM01.CH05,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR005,EG-AS-TR-TB10.TR005,TrackerType,EG-AS-TS10-I19-SM01-CH05,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
...,...,...,...,...,...,...,...
2933,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH06,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR145,EG-AS-TR-TB05.TR145,TrackerType,EG-AS-TS05-I10-SM13-CH06,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
2934,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH07,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR147,EG-AS-TR-TB05.TR147,TrackerType,EG-AS-TS05-I10-SM13-CH07,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
2935,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH08,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR148,EG-AS-TR-TB05.TR148,TrackerType,EG-AS-TS05-I10-SM13-CH08,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
2936,4:1:SSO.EG-AS.S1.Z6.TS05.I10.SM13.CH09,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR150,EG-AS-TR-TB05.TR150,TrackerType,EG-AS-TS05-I10-SM13-CH09,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."


time: 401 ms (started: 2022-08-26 13:18:40 +02:00)


In [20]:
track_var_list = ['AngleMeasured', 'AngleSetpoint']

time: 568 µs (started: 2022-08-26 13:18:41 +02:00)


In [21]:
# Live value trackers data
track_liv_values = opc.get_live_values_data(track_var_list, trackers)
track_liv_values

,Id,Name,Variable,Timestamp,Value
0,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR001.Signals.Angl...,EG-AS-TR-TB10.TR001,AngleMeasured,2022-08-26T11:16:23.7024553Z,20.364107
1,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR001.Signals.Angl...,EG-AS-TR-TB10.TR001,AngleSetpoint,2022-08-26T11:16:23.7024553Z,21.368660
2,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR002.Signals.Angl...,EG-AS-TR-TB10.TR002,AngleMeasured,2022-08-26T11:16:23.7024553Z,20.428648
3,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR002.Signals.Angl...,EG-AS-TR-TB10.TR002,AngleSetpoint,2022-08-26T11:16:23.7024553Z,21.409485
4,4:1:SSO.EG-AS.S1.M1.TR-TB10.TR004.Signals.Angl...,EG-AS-TR-TB10.TR004,AngleMeasured,2022-08-26T11:16:23.7024553Z,21.411800
...,...,...,...,...,...
5871,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR148.Signals.Angl...,EG-AS-TR-TB05.TR148,AngleSetpoint,2022-08-26T11:16:23.7024553Z,21.405325
5872,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR150.Signals.Angl...,EG-AS-TR-TB05.TR150,AngleMeasured,2022-08-26T11:16:23.7024553Z,20.364218
5873,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR150.Signals.Angl...,EG-AS-TR-TB05.TR150,AngleSetpoint,2022-08-26T11:16:23.7024553Z,21.409485
5874,4:1:SSO.EG-AS.S1.M1.TR-TB05.TR149.Signals.Angl...,EG-AS-TR-TB05.TR149,AngleMeasured,2022-08-26T11:16:23.7024553Z,21.733799


time: 7.11 s (started: 2022-08-26 13:18:43 +02:00)


In [22]:
# 1 day trackers data
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time, pro_interval, agg_name, trackers, track_var_list)
await session.close()

time: 1min 27s (started: 2022-08-26 13:18:52 +02:00)


In [24]:
# 15 day trackers data 
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time1, pro_interval, agg_name, trackers, track_var_list)
await session.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4166a1f2b0>


time: 21min 45s (started: 2022-08-26 13:27:26 +02:00)


In [26]:
# 1 month trackers data 
session = ClientSession()
await opc.get_agg_hist_value_data(session, start_time, end_time2, pro_interval, agg_name, trackers, track_var_list)
await session.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4147820f10>


time: 15min 39s (started: 2022-08-26 13:57:07 +02:00)
